In [1]:
import pandas as pd
import numpy as np

In [43]:
df = pd.read_csv('/Users/xuhuizhou/Projects/context-sbf/data/inference_data/toxigen_explanations_v2/toxigen_explanations_train.csv')
len(df)

31582

In [44]:
df.fillna('none', inplace=True)
len(df)

31582

In [45]:
df.to_csv('/Users/xuhuizhou/Projects/context-sbf/data/inference_data/toxigen_explanations_v2/toxigen_explanations_train.csv', index=False)

In [20]:
def get_offensiveness_binary(data):
    data.fillna(' ', inplace = True)
    off_data = data['offensiveness']
    offensive = []
    for i, row in enumerate(off_data.str.split(',', expand=False)):
        #if not isinstance(row, float):
        offensive.append(row[0])
            
    offensive = list(map(lambda x: x.lstrip().lower(), offensive))
    
    offensive = list(map(lambda x: x.replace('very ', ''), offensive))
    offensive = list(map(lambda x: x.replace('highly ', ''), offensive))
    offensive = list(map(lambda x: x.replace('extremely ', ''), offensive))
    
    for i, off in enumerate(offensive):
        if 'anti-sem' in off:
            offensive[i] = 'anti-semitic'
        if 'none' in off or 'not offensive' in off or \
             'not necessarily offensive' in off or 'n/a' in off or \
             'not applicable' in off:
            offensive[i] = 'not offensive'
        if 'ableis' in off:
            offensive[i] = 'ableist'
        if 'racis' in off:
            offensive[i] = 'racist'
        if 'stereotype' in off:
            offensive[i] = 'offensive stereotype'

    off_binary = (np.array(offensive) != 'not offensive').astype(int)
    return off_binary
    

In [21]:
off_binary = get_offensiveness_binary(df)
df['offensive_binary'] = off_binary

In [22]:
df['offensive_binary'].value_counts()

1    29032
0     3550
Name: offensive_binary, dtype: int64

In [28]:
df_nontoxic = df[df['offensive_binary']==0]
non_toxic_statements = set(df_nontoxic['statement'].values)
df_nontoxic_related = df.groupby("statement").filter(
            lambda x: x["statement"].values[0] in non_toxic_statements
        )

In [30]:
len(df_nontoxic_related)


5678

In [31]:
df_nontoxic_related.to_csv('/Users/xuhuizhou/Projects/context-sbf/data/inference_data/toxigen_explanations_v2/toxigen_explanations_nontoxic.csv', index=False)

In [29]:
df = pd.read_csv('/Users/xuhuizhou/Projects/context-sbf/data/inference_data/adversarial_contexts_statements/mAgr_contexts.csv')

In [30]:
# remove rows have the none values
print(len(df))
df = df.dropna()
Na_list = []
for col in df.columns:
    if col.split('_')[0] in ['harmless', 'offensive']:
        print(col)
        if Na_list==[]:
            Na_list = df[col].str.contains('none|N/A', case=False).tolist()
        else:
            Na_list = [x or y for x, y in zip(Na_list, df[col].str.contains('none|N/A', case=False).tolist())]
    print(sum(Na_list))
df = df[~np.array(Na_list)]
df.to_csv('/Users/xuhuizhou/Projects/context-sbf/data/inference_data/adversarial_contexts_statements/mAgr_contexts_cleaned.csv', index=False)


1522
0
0
0
0
0
harmless_situationalContext
63
harmless_speakerIdentity
63
harmless_listenerIdentity
63
offensive_situationalContext
65
offensive_speakerIdentity
65
offensive_listenerIdentity
65


## Transform the adversarial examples into a normal format

In [10]:
df = pd.read_csv('/Users/xuhuizhou/Projects/context-sbf/data/inference_data/adversarial_contexts_statements/mAgr_contexts_cleaned.csv')
print(len(df))

1272


In [7]:
df_harmless = df[['caption','tags', 'quote', 'examples', 'statement', 'harmless_situationalContext', 'harmless_speakerIdentity', 'harmless_listenerIdentity']]
df_offensive = df[['caption','tags', 'quote', 'examples', 'statement', 'offensive_situationalContext', 'offensive_speakerIdentity', 'offensive_listenerIdentity']]
df_harmless['moreOff'] = [-1]*len(df_harmless)
df_harmless.rename(columns={'harmless_situationalContext': 'situationalContext', 'harmless_speakerIdentity': 'speakerIdentity', 'harmless_listenerIdentity': 'listenerIdentity'}, inplace=True)
df_offensive['moreOff'] = [1]*len(df_offensive)
df_offensive.rename(columns={'offensive_situationalContext': 'situationalContext', 'offensive_speakerIdentity': 'speakerIdentity', 'offensive_listenerIdentity': 'listenerIdentity'}, inplace=True)
df_overall = pd.concat([df_harmless, df_offensive], axis=0)

/var/folders/gx/1s72qn6d30qc9jq9tywjml2m0000gn/T/ipykernel_58145/3696636120.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_harmless['moreOff'] = [-1]*len(df_harmless)
/var/folders/gx/1s72qn6d30qc9jq9tywjml2m0000gn/T/ipykernel_58145/3696636120.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_harmless.rename(columns={'harmless_situationalContext': 'situationalContext', 'harmless_speakerIdentity': 'speakerIdentity', 'harmless_listenerIdentity': 'listenerIdentity'}, inplace=True)
/var/folders/gx/1s72qn6d30qc9jq9tywjml2m0000gn/T/ipykernel_58145/369663612

In [11]:
df_overall
print(len(df_overall))
df_overall.to_csv('/Users/xuhuizhou/Projects/context-sbf/data/inference_data/adversarial_contexts_statements/mAgr_contexts_single.csv', index=False)

2544
